In [1]:
import init

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from scipy.stats import entropy

from dataset.graph_loader import GraphLoader

In [178]:
loader = GraphLoader(
#     data_path='/Users/marvzhang/projects/tikazyq/auto-html/data/quotes.toscrape.com/json/http___quotes_toscrape_com_.json', 
#     data_path='/Users/marvzhang/projects/tikazyq/auto-html/data/ssr1.scrape.center/json/https___ssr1_scrape_center.json', 
    data_path='/Users/marvzhang/projects/tikazyq/auto-html/data/docs.scrapy.org/json/https___docs_scrapy_org_en_latest_.json', 
    embed_walk_length=3,
)

loader.run()

Graph(num_nodes=555, num_edges=555,
      ndata_schemes={'feat': Scheme(shape=(233,), dtype=torch.float32)}
      edata_schemes={})


/Users/marvzhang/projects/tikazyq/auto-html/venv/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/Users/marvzhang/projects/tikazyq/auto-html/venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [179]:
loader.nodes_embedded_tensor

tensor([[280,   0, 412, 313],
        [291, 280,   0, 412],
        [302, 291, 280,   0],
        ...,
        [512, 510, 506, 428],
        [513, 280,   0, 412],
        [514, 280,   0, 412]])

In [180]:
idx = loader.nodes_embedded_tensor[10]
idx

tensor([390, 368, 357, 324])

In [181]:
loader.nodes_features_tensor[idx]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0.,

In [182]:
loader.nodes_features_tensor[idx].shape

torch.Size([4, 233])

In [183]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor].shape

torch.Size([555, 4, 233])

In [184]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

## Nodes Features (tags + attributes)

In [185]:
feat_tags_attrs = loader.nodes_features_tensor
feat_tags_attrs

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]], requires_grad=True)

In [186]:
X_feat_tags_attrs = feat_tags_attrs.detach().numpy()
X_feat_tags_attrs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [187]:
X_feat_tags_attrs = normalize(X_feat_tags_attrs, norm='l1', axis=1)
X_feat_tags_attrs

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0.5, 0. , 0. ]], dtype=float32)

## Nodes Features (node2vec)

In [188]:
feat_node2vec = loader.nodes_features_tensor[loader.nodes_embedded_tensor].sum(dim=1)
feat_node2vec.shape

torch.Size([555, 233])

In [189]:
X_feat_node2vec = feat_node2vec.detach().numpy()
X_feat_node2vec.shape

(555, 233)

In [190]:
X_feat_node2vec = normalize(X_feat_node2vec, norm='l1', axis=1)
X_feat_node2vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Nodes Features (all: tags + attributes + node2vec)

In [191]:
X = np.concatenate((X_feat_tags_attrs, X_feat_node2vec), axis=1)
X.shape

(555, 466)

In [192]:
X = normalize(X, norm='l2', axis=1)
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## DBSCAN Clustering

In [193]:
dbscan = DBSCAN()
dbscan.fit(X)

DBSCAN()

In [194]:
labels = pd.Series(dbscan.labels_)
labels_counts = labels.value_counts()
labels_counts[labels_counts.index > -1]

11    82
5     58
10    50
7     48
3     47
4     10
6      7
0      6
2      6
8      6
9      6
1      5
12     5
dtype: int64

In [195]:
for i in labels_counts[labels_counts.index > -1].index.tolist():
    top_label = labels_counts.index.values[i]
    top_label

    top_nodes_idx = labels[labels == top_label].index.values
    top_nodes_idx

    nodes = np.array(loader.nodes)[top_nodes_idx]
    nodes = sorted(nodes, key=lambda n: n.graph_node_id)
#     nodes = list(map(lambda n: f'[{loader.graphs_id_file_map.get(n.graph_id)}] {n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    nodes = list(map(lambda n: f'{n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    print(np.array(nodes[:50]))
    print('\n\n')

["0.224 -> 0.220: [('tag', 'dl'), ('class', 'simple')]"
 "0.250 -> 0.245: [('tag', 'dl'), ('class', 'simple')]"
 "0.317 -> 0.313: [('tag', 'dl'), ('class', 'simple')]"
 "0.342 -> 0.338: [('tag', 'dl'), ('class', 'simple')]"
 "0.431 -> 0.426: [('tag', 'dl'), ('class', 'simple')]"
 "0.481 -> 0.477: [('tag', 'dl'), ('class', 'simple')]"]



["0.103 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.105 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.107 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.109 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.111 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.113 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.115 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.117 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.119 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.121 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"
 "0.123 -> 0.102: [('tag', 'li'), ('class', 'toctree-l1')]"


## Filter Clusters

In [196]:
df_nodes = pd.DataFrame({
    'id': [n.id for n in loader.nodes],
    'parent_id': [n.parent_id for n in loader.nodes],
    'label': dbscan.labels_,
})
df_nodes

,id,parent_id,label
0,38,1,-1
1,39,38,-1
2,40,39,-1
3,41,40,-1
4,42,41,-1
...,...,...,...
550,588,587,-1
551,589,585,11
552,590,589,-1
553,591,38,-1


In [197]:
df_nodes = df_nodes[df_nodes.label != -1]
df_nodes.head()

,id,parent_id,label
13,51,50,0
14,52,51,1
15,62,50,0
16,63,62,1
17,89,50,0


In [198]:
df_labels = df_nodes \
    .groupby('label')[['parent_id']] \
    .agg(lambda s: entropy(s.value_counts())) \
    .rename(columns={'parent_id': 'entropy'}) \
    .sort_values(by='entropy', ascending=True) \
    .reset_index()
df_labels

,label,entropy
0,0,0.000000
1,2,0.000000
2,4,0.500402
3,1,1.609438
4,12,1.609438
5,3,1.624598
6,8,1.791759
7,9,1.791759
8,10,1.820006
9,11,1.829660


In [199]:
for label in df_labels[df_labels.entropy < 0.5].label.tolist():
    # mask
    mask = df_nodes.label == label
    
    # entropy
    arr = df_nodes[mask].parent_id
    prob = arr.value_counts(normalize=True)
    ent = df_labels[df_labels.label == label]
    
    # nodes
    nodes_ids = df_nodes[mask].id.values
    nodes = np.array([n for n in loader.nodes if n.id in nodes_ids])
    
    # results
    print(f'label: {label}')
    print(f'entropy: {ent}')
    print(df_nodes[mask].parent_id.values)
    print(nodes)
    print('\n')

label: 0
entropy:    label  entropy
0      0      0.0
[50 50 50 50 50 50]
[{'id': 51, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}
 {'id': 62, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}
 {'id': 89, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}
 {'id': 100, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}
 {'id': 133, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}
 {'id': 154, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'p'), ('class', 'caption'), ('role', 'heading')]}]


label: 2
entropy:    label  entropy
1      2      0.0
[50 50 50 50 50 50]
[{'id': 53, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'ul')]}
 {'id': 64, 'parent_id': 50, 'graph_id': 0, 'features': [('tag', 'ul')]}
 {'id': 91